In [1]:
"""import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        """

"import tensorflow as tf\ngpus = tf.config.list_physical_devices('GPU')\nif gpus:\n    try:\n        for gpu in gpus:\n            tf.config.experimental.set_memory_growth(gpu, True)\n    except RuntimeError as e:\n        print(e)\n        "

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3072)])  # 4GB limit
    except RuntimeError as e:
        print(e)


2025-06-20 18:45:53.938299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750445153.964169   13217 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750445153.971623   13217 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750445153.995243   13217 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750445153.995322   13217 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750445153.995324   13217 computation_placer.cc:177] computation placer alr

In [3]:
NUM_SENTENCES = 4500 # Use only the first 20,000 records.
MAX_NUM_WORDS = 10000 # Use 20,000 words for tokenizing
MAX_SENT_LEN = 150

EMBEDDING_SIZE = 100

LSTM_NEURONS = 300

BATCH_SIZE = 32
EPOCHS = 50



In [4]:
import csv
def read_text_file(filename): 
    lines = []
    try:
        with open(filename, newline='\n', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile)
            for row in csv_reader:
              lines.append(row)
        #print(len(lines))
        #print(lines[0])
        return lines
    except FileNotFoundError:
        return f"Error: The file '{filename}' was not found."
    except Exception as e:
        return f"An error occurred: {e}"
    

file_name = '/mnt/d/Work/wsl/ML-DS/english_to_bangla.csv'
# Read the content of the file
lines = read_text_file(file_name)
 

In [5]:
lines[1]

['a child in a pink dress is climbing up a set of stairs in an entry way .',
 'একটি গোলাপী জামা পরা বাচ্চা মেয়ে একটি বাড়ির প্রবেশ পথের সিঁড়ি বেয়ে উঠছে।']

In [6]:
inputs = []
outputs_i = []
outputs = []

 
count = 0
for line in lines: 
    if count > NUM_SENTENCES:
        break
    count += 1 
    ip = line[0].rstrip()
    temp_op = line[1].rstrip()
    op_i = '<sos> '+temp_op
    op = temp_op+' <eos>'
    inputs.append(ip)
    outputs_i.append(op_i)
    outputs.append(op)

print('Total inputs =', len(inputs))
print('Total intermediate outputs =', len(outputs_i))
print('Total outputs =', len(outputs))
print('\nSample:')
print(inputs[10])
print(outputs_i[10])
print(outputs[10])

Total inputs = 5001
Total intermediate outputs = 5001
Total outputs = 5001

Sample:
two dogs on pavement moving toward each other .
<sos> রাস্তার পাশে দুইটি কুকুর পরস্পরের দিকে এগিয়ে যাচ্ছ।
রাস্তার পাশে দুইটি কুকুর পরস্পরের দিকে এগিয়ে যাচ্ছ। <eos>


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(inputs)

inputs_seq = input_tokenizer.texts_to_sequences(inputs)

inputs_word2index = input_tokenizer.word_index
print('Total unique words in input:', len(inputs_word2index))

inputs_numwords = len(inputs_word2index)+1

inputs_maxlen = max(len(s) for s in inputs_seq)
print('Length of longest sentence in input:', inputs_maxlen)

output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(outputs_i + outputs)

outputs_i_seq = output_tokenizer.texts_to_sequences(outputs_i)
outputs_seq = output_tokenizer.texts_to_sequences(outputs)

outputs_word2index = output_tokenizer.word_index
print('Total unique words in output:', len(outputs_word2index))

outputs_numwords = len(outputs_word2index)+1

outputs_maxlen = max(len(s) for s in outputs_seq)
print('Length of longest sentence in output:', outputs_maxlen)

Total unique words in input: 3230
Length of longest sentence in input: 33
Total unique words in output: 5375
Length of longest sentence in output: 33


In [8]:
from keras.preprocessing.sequence import pad_sequences

encoder_input_sequences = pad_sequences(inputs_seq, maxlen=inputs_maxlen)
print('encoder_input_sequences shape:', encoder_input_sequences.shape)

decoder_input_sequences = pad_sequences(outputs_i_seq, maxlen=outputs_maxlen, padding='post')
print('decoder_inputs_sequences shape:', decoder_input_sequences.shape)

decoder_output_sequences = pad_sequences(outputs_seq, maxlen=outputs_maxlen, padding='post')
print('decoder_output_sequences shape:', decoder_output_sequences.shape)



encoder_input_sequences shape: (5001, 33)
decoder_inputs_sequences shape: (5001, 33)
decoder_output_sequences shape: (5001, 33)


In [9]:
from numpy import asarray, zeros
 
embeddings_dict = dict()
glove_file = open(f'/mnt/d/Work/wsl/ML-DS/glove.6B.{EMBEDDING_SIZE}d.txt', encoding='utf-8') 

for line in glove_file:
    records = line.split()
    word = records[0]
    vector = asarray(records[1:], dtype='float32')
    embeddings_dict[word] = vector

glove_file.close()

num_words = min(MAX_NUM_WORDS, len(inputs_word2index)+1)

embedding_matrix = zeros((num_words, EMBEDDING_SIZE))

for word, index in inputs_word2index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [10]:
from keras.layers import Input, Embedding, LSTM, Dense

model_mode = 1

if (model_mode == 1) :
    encoder_embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=inputs_maxlen)
    decoder_embedding_layer = Embedding(outputs_numwords, LSTM_NEURONS)
    
    decoder_outputs_onehot = zeros((len(inputs), outputs_maxlen, outputs_numwords), dtype='float32')
    print('decoder_outputs_onehot shape:', decoder_outputs_onehot.shape)
    
    for i, d in enumerate(decoder_output_sequences):
        for t, w in enumerate(d):
            decoder_outputs_onehot[i, t, w] = 1
    
    encoder_inputs = Input(shape=(inputs_maxlen,))
    encoder_inputs_emb = encoder_embedding_layer(encoder_inputs)
    encoder = LSTM(LSTM_NEURONS, return_state=True)
    encoder_outputs, h, c = encoder(encoder_inputs_emb)
    encoder_states = [h, c]
    
    decoder_inputs = Input(shape=(outputs_maxlen,))
    decoder_inputs_emb = decoder_embedding_layer(decoder_inputs)
    decoder = LSTM(LSTM_NEURONS, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder(decoder_inputs_emb, initial_state=encoder_states)
    
    output_dense_layer = Dense(outputs_numwords, activation='softmax')
    outputs = output_dense_layer(decoder_outputs)

/home/sid/miniconda3/envs/tensorflow-2.19_new/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1750445175.707398   13217 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3072 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


decoder_outputs_onehot shape: (5001, 33, 5376)


In [11]:
from keras.models import Model

model = Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 33)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 33)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 33, 100)   │    323,100 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 33, 300)   │  1,612,800 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 300),     │    481,200 │ embedding[0][0]   │
│                     │ (None, 300),      │            │                   │
│                     │ (None, 300)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 33, 300), │    721,200 │ embedding_1[0][0… │
│                     │ (None, 300),      │            │ lstm[0][1],       │
│                     │ (None, 300)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 33, 5376)  │  1,618,176 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,756,476 (18.14 MB)

 Trainable params: 4,756,476 (18.14 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping 
early_stopping_callback = EarlyStopping(monitor='val_loss', 
                                        min_delta=0.001,
                                        patience=5,
                                        restore_best_weights=True, 
                                        verbose=1,
                                        mode='auto')
trn = model.fit([encoder_input_sequences, decoder_input_sequences],
               decoder_outputs_onehot, 
               batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2,
                callbacks=[early_stopping_callback]
               )


In [ ]:

save_path = 'NUM_SENTENCES_' + str(NUM_SENTENCES) + "_EMBEDDING_SIZE_" + str(EMBEDDING_SIZE) + '_LSTM_NEURONS_' + str(LSTM_NEURONS) + '_model_mode_' + str(model_mode) 
folder_name = '/mnt/d/Work/wsl/ML-DS/saved_models/EnglishtoBengaliForMachineTranslation/model/' + save_path
print(folder_name)
# Save to a directory in the TensorFlow SavedModel format (recommended)
#model.save(folder_name+"/model.keras")

# Or save as a single HDF5 file (legacy Keras format)
#model.save(folder_name + '/model.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
print(encoder_model.summary())

In [ ]:
decoder_input_h = Input(shape=(LSTM_NEURONS,))
decoder_input_c = Input(shape=(LSTM_NEURONS,))
decoder_input_states = [decoder_input_h, decoder_input_c]

decoder_input_word = Input(shape=(1,))
decoder_input_word_emb = decoder_embedding_layer(decoder_input_word)

decoder_outputs, h, c = decoder(decoder_input_word_emb, initial_state=decoder_input_states)
decoder_states = [h, c]

outputs = output_dense_layer(decoder_outputs)


decoder_model = Model([decoder_input_word]+decoder_input_states, [outputs]+decoder_states)
print(decoder_model.summary())

In [ ]:
from numpy import argmax

index_to_word_input = {v:k for k,v in inputs_word2index.items()}
index_to_word_output = {v:k for k,v in outputs_word2index.items()}

def translate(input_seq):
    states = encoder_model.predict(input_seq)
    
    sos = outputs_word2index['<sos>']
    eos = outputs_word2index['<eos>']
    
    output_seq = zeros((1, 1))
    output_seq[0, 0] = sos
    
    output_sentence = []
    
    for _ in range(outputs_maxlen):
        output_tokens, h, c = decoder_model.predict([output_seq]+states)
        idx = argmax(output_tokens[0, 0, :])
        
        if idx == eos:
            break     
        word = ''
        if idx > 0:
            word = index_to_word_output[idx]
            output_sentence.append(word)
        
        states = [h, c]
        output_seq[0, 0] = idx
    
    return ' '.join(output_sentence)

In [ ]:
from numpy import random

i = random.choice(len(inputs))
input_seq = encoder_input_sequences[i:i+1]
translation = translate(input_seq)

print('Input:', inputs[i])
print('Response:', translation)



In [ ]:
my_inputs = ['I am good']
my_inputs_seq = input_tokenizer.texts_to_sequences(my_inputs)
my_encoder_input_sequences = pad_sequences(my_inputs_seq, maxlen=inputs_maxlen)

translation = translate(my_encoder_input_sequences[0:1])
print(translation)